In [2]:
# Title:     SmartHome Dataset
# Authors:   Aline J
# Date:      01/06/2020
# Goal:      Analyze and predict energy use efficiency 
#            in a smarthouse during a year under variable weather conditions.

# Questions: 
# 1) What are the variables that control generated energy? 
# 2) What are the variables that control used energy? 
# 3) What are the variables that control energy efficiency?

# Method. Unsupervised Machine learning Method, specifically random forest regression analysis.
# Results: Used CleanData
#         Several attempts to run the analysis were done at differen frequencies. 
#         Only Monthly frequency data resulted the highest accuracy (76%) vs Minute data (0.01%).  
#         The minimum random state of 50, were enough to explain the variability in the dataset.  
#         The same accuracy was yield when running random state of 1000.


In [3]:
### Begins Here ####
# 1. Import dependencies
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
# Other dependencies
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy import stats
import string as string
import warnings
warnings.filterwarnings('ignore')

In [45]:
# 2. Import Raw data file into a data frame
csv_path = "Data/HomeC.csv"
my_data = pd.read_csv(csv_path,   parse_dates=True)
home_dat = my_data.select_dtypes(exclude=['object'])
nRow, nCol = my_data.shape
print(f'There are {nRow} rows and {nCol} columns')


There are 503911 rows and 32 columns


In [46]:
home_dat.head(5)
home_dat.tail(5)


,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
503906,1.599333,0.003233,1.599333,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,0.013433,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503907,1.924267,0.003217,1.924267,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503908,1.978200,0.003217,1.978200,0.000050,0.495667,0.620367,0.042100,0.005333,0.008233,0.012817,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503909,1.990950,0.003233,1.990950,0.000050,0.494700,0.634133,0.042100,0.004917,0.008133,0.012833,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
503910,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
time_index = pd.date_range('2016-01-01 05:00', periods=503911,  freq='min')  
time_index = pd.DatetimeIndex(time_index)
home_dat = home_dat.set_index(time_index)
# Check that a dataset has been uploaded into my_data

In [48]:
home_dat.head(5)
home_dat.tail(5)

,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
2016-12-16 03:26:00,1.599333,0.003233,1.599333,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,0.013433,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
2016-12-16 03:27:00,1.924267,0.003217,1.924267,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
2016-12-16 03:28:00,1.978200,0.003217,1.978200,0.000050,0.495667,0.620367,0.042100,0.005333,0.008233,0.012817,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
2016-12-16 03:29:00,1.990950,0.003233,1.990950,0.000050,0.494700,0.634133,0.042100,0.004917,0.008133,0.012833,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
2016-12-16 03:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
home_dat.columns = [col.replace(' [kW]', '') for col in home_dat.columns]
home_dat.columns

Index(['use', 'gen', 'House overall', 'Dishwasher', 'Furnace 1', 'Furnace 2',
       'Home office', 'Fridge', 'Wine cellar', 'Garage door', 'Kitchen 12',
       'Kitchen 14', 'Kitchen 38', 'Barn', 'Well', 'Microwave', 'Living room',
       'Solar', 'temperature', 'humidity', 'visibility', 'apparentTemperature',
       'pressure', 'windSpeed', 'windBearing', 'precipIntensity', 'dewPoint',
       'precipProbability'],
      dtype='object')

In [50]:
home_dat = home_dat.dropna(axis='columns', how='all')

In [51]:
home_dat = home_dat[0:-1] ## == dataset[0:dataset.shape[0]-1] == dataset[0:len(dataset)-1] == dataset[:-1]
home_dat.tail(3)


,use,gen,House overall,Dishwasher,Furnace 1,Furnace 2,Home office,Fridge,Wine cellar,Garage door,...,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability
2016-12-16 03:27:00,1.924267,0.003217,1.924267,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
2016-12-16 03:28:00,1.978200,0.003217,1.978200,0.000050,0.495667,0.620367,0.042100,0.005333,0.008233,0.012817,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51
2016-12-16 03:29:00,1.990950,0.003233,1.990950,0.000050,0.494700,0.634133,0.042100,0.004917,0.008133,0.012833,...,35.12,0.86,8.74,29.45,1011.49,6.72,186.0,0.0101,31.27,0.51


In [52]:
home_dat['Furnace'] = home_dat[['Furnace 1','Furnace 2']].sum(axis=1)
home_dat['Kitchen'] = home_dat[['Kitchen 12','Kitchen 14','Kitchen 38']].sum(axis=1)

#df['Result'] = df['Column A']/df['Column B']
home_dat.columns

Index(['use', 'gen', 'House overall', 'Dishwasher', 'Furnace 1', 'Furnace 2',
       'Home office', 'Fridge', 'Wine cellar', 'Garage door', 'Kitchen 12',
       'Kitchen 14', 'Kitchen 38', 'Barn', 'Well', 'Microwave', 'Living room',
       'Solar', 'temperature', 'humidity', 'visibility', 'apparentTemperature',
       'pressure', 'windSpeed', 'windBearing', 'precipIntensity', 'dewPoint',
       'precipProbability', 'Furnace', 'Kitchen'],
      dtype='object')

In [53]:
DescriptiveStats=home_dat.describe()
DescriptiveStats.head()

,use,gen,House overall,Dishwasher,Furnace 1,Furnace 2,Home office,Fridge,Wine cellar,Garage door,...,visibility,apparentTemperature,pressure,windSpeed,windBearing,precipIntensity,dewPoint,precipProbability,Furnace,Kitchen
count,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,...,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000,503910.000000
mean,0.858962,0.076229,0.858962,0.031368,0.099210,0.136779,0.081287,0.063556,0.042137,0.014139,...,9.253444,48.263382,1016.301625,6.649936,202.356843,0.002598,38.694013,0.056453,0.235989,0.009788
std,1.058207,0.128428,1.058207,0.190951,0.169059,0.178631,0.104466,0.076199,0.057967,0.014292,...,1.611186,22.027916,7.895185,3.982716,106.520474,0.011257,19.087939,0.165836,0.273885,0.079883
min,0.000000,0.000000,0.000000,0.000000,0.000017,0.000067,0.000083,0.000067,0.000017,0.000017,...,0.270000,-32.080000,986.400000,0.000000,0.000000,0.000000,-27.240000,0.000000,0.000300,0.000000
25%,0.367667,0.003367,0.367667,0.000000,0.020233,0.064400,0.040383,0.005083,0.007133,0.012733,...,9.420000,31.090000,1011.290000,3.660000,148.000000,0.000000,24.600000,0.000000,0.085300,0.000600


In [54]:
CleanData = home_dat.filter(items=[ 'gen', 'use', 'House overall', 
                               'Furnace', 'Home office',  'Living room',
                               'Wine cellar', 'Garage door', 'Kitchen',
                               'Barn', 'Well', 'Fridge', 'Microwave', 'Dishwasher',
                               'temperature', 'humidity', 
                               'visibility', 'apparentTemperature',
                               'pressure', 'windSpeed', 'windBearing', 'precipIntensity', 'dewPoint'])

In [55]:
WeatherData= home_dat.filter(items=[ 'gen', 'temperature', 'humidity', 
                               'visibility', 'apparentTemperature',
                               'pressure', 'windSpeed', 'windBearing', 'precipIntensity', 'dewPoint'])

In [56]:
RoomsData = home_dat.filter(items=[ 'gen', 'House overall', 
                               'Furnace', 'Home office',  'Living room',
                               'Wine cellar', 'Garage door', 'Kitchen',
                               'Barn', 'Well', 'Fridge', 'Microwave', 'Dishwasher'])

In [57]:
home_dat['EneEffi'] = home_dat['use']-home_dat['gen']

In [58]:
EnergyData = home_dat.filter(items=['EneEffi', 'gen', 'use', 'House overall'])

In [59]:
EnergyData.tail(3)

,EneEffi,gen,use,House overall
2016-12-16 03:27:00,1.921050,0.003217,1.924267,1.924267
2016-12-16 03:28:00,1.974983,0.003217,1.978200,1.978200
2016-12-16 03:29:00,1.987717,0.003233,1.990950,1.990950


In [60]:
# Summaries
# Generate Data per (day) and (month) 
CleanData_per_day = CleanData.resample('D').sum()
WeatherData_per_day = WeatherData.resample('D').sum()
RoomsData_per_day = WeatherData.resample('D').sum()
EnergyData_per_day = WeatherData.resample('D').sum()

CleanData_per_month = CleanData.resample('M').sum()
WeatherData_per_month = WeatherData.resample('M').sum()
RoomsData_per_month = WeatherData.resample('M').sum()
EnergyData_per_month = WeatherData.resample('M').sum()




In [22]:
# Save Data to Directory
CleanData.to_csv(r'CleanData.csv')
EnergyData.to_csv(r'EnergyData.csv')
RoomsData.to_csv(r'RoomsData.csv')
WeatherData.to_csv(r'WeatherData.csv')

In [23]:
# Data Preparation
CleanData_per_month.columns
CleanData_per_month.dtypes 
#CleanData.tail()

gen                    float64
use                    float64
House overall          float64
Furnace                float64
Home office            float64
Living room            float64
Wine cellar            float64
Garage door            float64
Kitchen                float64
Barn                   float64
Well                   float64
Fridge                 float64
Microwave              float64
Dishwasher             float64
temperature            float64
humidity               float64
visibility             float64
apparentTemperature    float64
pressure               float64
windSpeed              float64
windBearing            float64
precipIntensity        float64
dewPoint               float64
dtype: object

In [24]:
# Running RF with all the data to investigate the driving variables of energy efficiency at a 1min resolution
#my_data.dtypes 
features=CleanData_per_month
type(features)
#features.iloc[:,5:].head(5)
#features.head(5)
#features.info()
features.columns

Index(['gen', 'use', 'House overall', 'Furnace', 'Home office', 'Living room',
       'Wine cellar', 'Garage door', 'Kitchen', 'Barn', 'Well', 'Fridge',
       'Microwave', 'Dishwasher', 'temperature', 'humidity', 'visibility',
       'apparentTemperature', 'pressure', 'windSpeed', 'windBearing',
       'precipIntensity', 'dewPoint'],
      dtype='object')

In [25]:
# Labels are the values we want to predict
labels = np.array(features['gen'])
# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop('gen', axis = 1)
# Saving feature names for later use
feature_list = list(features.columns)
# Convert to numpy array
features = np.array(features)

In [26]:
# Training and Testing Sets
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 100)

In [27]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (9, 22)
Training Labels Shape: (9,)
Testing Features Shape: (3, 22)
Testing Labels Shape: (3,)


In [28]:
feature_list.index

<function list.index(value, start=0, stop=9223372036854775807, /)>

In [29]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 10 decision trees
#rf = RandomForestRegressor(n_estimators = 100, random_state = 100)
rf = RandomForestRegressor(n_estimators = 100, random_state = 100)
# Train the model on training data
rf.fit(train_features, train_labels);

In [30]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
#Mean Absolute Error: 8512.05 degrees

Mean Absolute Error: 540.33 degrees.


In [31]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)
# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')
#Accuracy: 95.96 %.
    # That looks pretty good! Our model has learned how to predict the List price as a function of list and close price in Houston with 95.96% accuracy.

Accuracy: 78.79 %.


In [61]:
plot = all_energy_effi.plot(kind = "bar", figsize = (5,5))
plot.set_title("energy efficiency")

NameError: name 'all_energy_effi' is not defined

In [35]:
!pip install pydot
!pip install graphviz


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 47.0/47.0 kB 138.3 kB/s eta 0:00:00



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
# What are the variables that regulate power generation?
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]

In [37]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot
# Pull out one tree from the forest
tree = rf.estimators_[5]

In [40]:
# Export the image to a dot file
export_graphviz(tree, out_file = 'tree', feature_names = feature_list, rounded = True, precision = 1)
# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree')
# Write graph to a png file
graph.write_png('tree.png')

FileNotFoundError: [WinError 2] "dot" not found in path.

In [41]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: temperature          Importance: 0.16
Variable: dewPoint             Importance: 0.16
Variable: Wine cellar          Importance: 0.11
Variable: apparentTemperature  Importance: 0.09
Variable: Garage door          Importance: 0.07
Variable: Kitchen              Importance: 0.04
Variable: pressure             Importance: 0.04
Variable: use                  Importance: 0.03
Variable: Living room          Importance: 0.03
Variable: Fridge               Importance: 0.03
Variable: Microwave            Importance: 0.03
Variable: Dishwasher           Importance: 0.03
Variable: visibility           Importance: 0.03
Variable: precipIntensity      Importance: 0.03
Variable: Home office          Importance: 0.02
Variable: Barn                 Importance: 0.02
Variable: windSpeed            Importance: 0.02
Variable: windBearing          Importance: 0.02
Variable: House overall        Importance: 0.01
Variable: Furnace              Importance: 0.01
Variable: Well                 Importanc

In [42]:
plt.figure();
feature_importances.plot();

AttributeError: 'list' object has no attribute 'plot'

<Figure size 640x480 with 0 Axes>

In [43]:
# New random forest with only the two most important variables
rf_most_important = RandomForestRegressor(n_estimators= 1000, random_state=1000)
# Extract the two most important features
important_indices = [feature_list.index('temperature'), feature_list.index('dewPoint')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]
# Train the random forest
rf_most_important.fit(train_important, train_labels)
# Make predictions and determine the error
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - test_labels)
# Display the performance metrics
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape
print('Accuracy:', round(accuracy, 2), '%.')

#Note:
#This tells us that temperature and dew point are the two most important variables to regulate generated energy 

Mean Absolute Error: 590.74 degrees.
Accuracy: 76.45 %.
